In [1]:
from detectron2.data import DatasetCatalog, MetadataCatalog
from detectron2.data.datasets import register_coco_instances
register_coco_instances("bills", {}, "./annotations/instances_default.json", "./images")

In [2]:
dataset_dicts = DatasetCatalog.get("bills")
bills_metadata = MetadataCatalog.get("bills")

In [3]:
dataset_dicts

[{'file_name': './images/mcocr_warmup_fff96d88b5a9dfcd2c39c3e87ce1287e_00034.jpg',
  'height': 1024,
  'width': 768,
  'image_id': 1,
  'annotations': [{'iscrowd': 0,
    'bbox': [135.1, 56.85, 456.67, 894.45],
    'category_id': 0,
    'segmentation': [[150.88,
      56.85,
      288.36,
      66.34,
      424.26,
      67.92,
      591.77,
      67.92,
      577.55,
      949.69,
      395.82,
      944.95,
      135.1,
      951.3,
      138.24,
      485.1,
      146.14,
      102.68]],
    'bbox_mode': <BoxMode.XYWH_ABS: 1>}]},
 {'file_name': './images/mcocr_warmup_ffb7a6262cc068103b484bf8c0a11d93_00450.jpg',
  'height': 1820,
  'width': 1024,
  'image_id': 2,
  'annotations': [{'iscrowd': 0,
    'bbox': [16.2, 258.31, 853.83, 1443.64],
    'category_id': 0,
    'segmentation': [[24.63,
      297.63,
      297.07,
      258.31,
      448.73,
      258.31,
      603.21,
      258.31,
      870.03,
      308.86,
      830.71,
      586.92,
      811.05,
      1008.22,
      811.05,


In [ ]:
import random
import cv2
from detectron2.utils.visualizer import Visualizer

for d in random.sample(dataset_dicts, 3):
    print(d)
    img = cv2.imread(d["file_name"])
    visualizer = Visualizer(img[:, :, ::-1], metadata=bills_metadata, scale=0.5)
    vis = visualizer.draw_dataset_dict(d)
    cv2.imshow("Random", vis.get_image()[:, :, ::-1])
    cv2.waitKey(0)
cv2.destroyAllWindows()

In [4]:
from detectron2.engine import DefaultTrainer
from detectron2.config import get_cfg
import os

In [10]:
cfg = get_cfg()
cfg.merge_from_file(
    "./configs/COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"
)
cfg.DATASETS.TRAIN = ("bills",)
cfg.DATASETS.TEST = ()  # no metrics implemented for this dataset
cfg.DATALOADER.NUM_WORKERS = 2
cfg.MODEL.WEIGHTS = "detectron2://COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x/137849600/model_final_f10217.pkl"  # initialize from model zoo
cfg.SOLVER.IMS_PER_BATCH = 2
cfg.SOLVER.BASE_LR = 0.02
cfg.SOLVER.MAX_ITER = (
    300
)  # 300 iterations seems good enough, but you can certainly train longer
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = (
    128
)  # faster, and good enough for this toy dataset
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 1  # 3 classes (data, fig, hazelnut)

os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)

In [ ]:
trainer = DefaultTrainer(cfg)
trainer.resume_or_load(resume=False)
trainer.train()

In [4]:
import os
import random
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
cfg.DATALOADER.NUM_WORKERS = 2
cfg.MODEL.WEIGHTS = "detectron2://COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x/137849600/model_final_f10217.pkl"  # initialize from model zoo
cfg.SOLVER.IMS_PER_BATCH = 2
cfg.SOLVER.MAX_ITER = (
    300
)  # 300 iterations seems good enough, but you can certainly train longer
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = (
    128
)  # faster, and good enough for this toy dataset
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 1  # 3 classes (data, fig, hazelnut)
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.8   # set the testing threshold for this model
cfg.DATASETS.TEST = ("bills", )
predictor = DefaultPredictor(cfg)

In [5]:
import cv2
from detectron2.utils.visualizer import ColorMode
from detectron2.utils.visualizer import Visualizer
im = cv2.imread("/home/tuan/Documents/mcocr_public_train_test_shared_data/mcocr_val_data/val_images/mcocr_val_145115gozuc.jpg")
outputs = predictor(im)
v = Visualizer(im[:, :, ::-1],
                   scale=0.8,
                   instance_mode=ColorMode.IMAGE_BW   # remove the colors of unsegmented pixels
)
v = v.draw_instance_predictions(outputs["instances"].to("cpu"))
cv2.imshow("haha", v.get_image()[:, :, ::-1])
cv2.waitKey(0)
cv2.destroyAllWindows()

[ WARN:0@85.867] global /io/opencv/modules/imgcodecs/src/loadsave.cpp (239) findDecoder imread_('/home/tuan/Documents/mcocr_public_train_test_shared_data/mcocr_val_data/val_images/mcocr_val_145115gozuc.jpg'): can't open/read file: check file path/integrity


NameError: name 'predictor' is not defined

In [12]:
import torch
from detectron2.modeling import build_model

image = cv2.imread(
    '/home/tuan/Documents/mcocr_public_train_test_shared_data/mcocr_val_data/val_images/mcocr_val_145115gozuc.jpg')
height, width = image.shape[:2]
image = torch.as_tensor(image.astype("float32").transpose(2, 0, 1))
inputs = [{"image": image, "height": height, "width": width}]
model = build_model(cfg)
with torch.no_grad():
    images = model.preprocess_image(inputs)  # don't forget to preprocess
    features = model.backbone(images.tensor)
    proposals, _ = model.proposal_generator(images, features)
    instances, _ = model.roi_heads(images, features, proposals)
    mask_features = [features[f] for f in model.roi_heads.in_features]
    mask_features = model.roi_heads.mask_pooler(mask_features, [x.pred_boxes for x in instances])
    print(mask_features)

AssertionError: RPN requires gt_instances in training!